In [2]:
#https://youtu.be/1O_BenficgE?si=JqMGwh3s_FTC8k5d
import math
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

Matplotlib is building the font cache; this may take a moment.


In [3]:
sp500  = yf.Ticker("^GSPC")
sp500= sp500.history(period='max')

#sp500.plot.line(y='Close',use_index=True)

del sp500['Dividends']
del sp500['Stock Splits']

daysago = pd.DataFrame(dtype=object)
for i in range(1,90):
  daysago['{}dayago'.format(i)] = sp500['Close'].shift(i)
  #daysago['{}'.format(i)] = i
  #daysago.at['{}dayago'.format(i)] = daysago['{}'.format(i)],daysago['{}dayago'.format(i)]
#print(daysago)

sp500 = pd.concat([sp500,daysago],axis=1)

predictors = []

for i in range(1,90):
  predictors.append('{}dayago'.format(i))

predictors.append("Close")

sp500=sp500.reindex(columns=predictors)

sp500['tmr'] = sp500['Close'].shift(-1)
sp500['target'] = (sp500['tmr'] > sp500['Close']).astype(int)
sp500 = sp500.drop(['tmr'],axis=1)
#sp500['Close'] = 0,sp500['Close']
sp500 = sp500.loc['1990-01-01':].copy()

#print(sp500)
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=1000, min_samples_split=100,random_state=1)


shiii = - np.ceil(len(sp500)*0.2).astype(int)
train = sp500.iloc[:shiii]
test = sp500.iloc[shiii:]




#print(sp500)
model.fit(train[predictors], train["target"])

from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["target"], preds)
#0.5438292964244521
#0.5438292964244521
#0.5438292964244521
#0.5438292964244521
#after some weird shit idk
#0.5440922190201729

0.625

In [4]:
def predict(train,test,predictors,model):
  model.fit(train[predictors], train["target"])
  preds = model.predict(test[predictors])
  preds = pd.Series(preds, index=test.index,name='predicts')
  combined = pd.concat([test['target'],preds],axis=1)
  return combined


def backtest(data,model,predictors,start=2500,step=250):
  all_pred= []

  for i in range(start, data.shape[0], step):
      train = data.iloc[0:i].copy()
      test = data.iloc[i:(i+step)].copy()
      predictions = predict(train, test, predictors, model)
      all_pred.append(predictions)

  return pd.concat(all_pred)

In [5]:
predictions = backtest(sp500,model,predictors)

In [ ]:
precision_score(predictions["target"], predictions['predicts'])

0.5373348650201034

In [ ]:
data = {
    'weight': [0, 2],
    'height': [1, (3, 4)]
}
labels = ['cat', 'dog']
df = pd.DataFrame(data=data, index=labels)
df['e'] = df['height']
df